# Hexパッケージ

Elixirには[Hex](https://hex.pm/)というパッケージマネージャーがある。

Elixirでは`mix`というツールでプロジェクト管理をするのだが、[このように](https://hex.pm/docs/usage)プロジェクトの定義ファイルに依存パッケージを記述することでパッケージを利用可能になる。

HexパッケージにはErlang向けのものもあるが、それらはElixirでも問題なく使用することができる。

ここではnotebook上でいくつかのHexパッケージをインストールし、利用してみる。

- [Jason](https://hexdocs.pm/jason/readme.html): 高速なJSONパーサー・ジェネレーター
- [Croma](https://hexdocs.pm/croma/api-reference.html): Antikythera創始者の方が作成した、型ベースプログラミングのためのマクロ集
- [meck](https://hexdocs.pm/meck/): Erlang向けのモッキングライブラリ

[Antikythera](https://hexdocs.pm/antikythera/api-reference.html)もHexパッケージとして公開されている。
(Jupyter notebook上で動作させることは難しいので、ここでは扱わない)

## Jupyter notebook上での利用

通常依存パッケージはmix projectで管理するが、ここでは動的にパッケージのインストールと利用を行うため、[Boyle](https://github.com/pprzetacznik/IElixir#package-management-with-boyle)というモジュールを利用する、

あくまでもJupyter notebook上で動作させる場合に特有の事情である。

In [ ]:
Boyle.mk("training_env")
Boyle.activate("training_env")
# :ok が返ること

In [ ]:
Boyle.install({:jason, "1.2.2"})
Boyle.install({:croma, "0.10.2"})
Boyle.install({:meck, "0.9.2"})
# :ok が返ること

### Jason

- `Jason.decode/1` でJSON形式の文字列をElixirの値にパースする
  - `Jason.decode!/1` はパースに失敗するとエラー
- `Jason.encode/1` でElixirの値をJSON文字列にエンコードする
  - `Jason.encode!/1` はエンコードに失敗するとエラー
  
WebサーバーではHTTPリクエストやレスポンスのbodyに対して適用することが多い。

In [ ]:
json = """
{
  "x": 0,
  "y": "a",
  "array": [0, 1, 2],
  "nested": {
    "inner": {}
  }
}
"""

{:ok, map} = Jason.decode(json)
IO.inspect map

In [ ]:
File.read!("./resources/7/test.json")
|> Jason.decode!()

In [ ]:
{:error, reason} = Jason.decode("{[]}")

In [ ]:
result = Jason.decode!("{[]}")
# => Jason.DecodeError

In [ ]:
{:ok, json} = Jason.encode(%{a: 0})
IO.puts json

### Croma

- Elixirで型ベースプログラミングを行うのを楽にするマクロ集


In [ ]:
# Croma.Structを使ってStructを定義
# new/1 または new!/1 関数を使ってStructを生成する時、フィールドのvalidationが自動で行われる


defmodule TestStruct do
  use Croma.Struct, recursive_new?: true, fields: [
    x: Croma.Integer
  ]
end

defmodule OtherStruct do
  use Croma.Struct, recursive_new?: true, fields: [
    x: Croma.Integer
  ]
end

In [ ]:
TestStruct.new(%{x: 0})

In [ ]:
TestStruct.new(%{x: 1.0})

In [ ]:
%TestStruct{x: x} = TestStruct.new!(%{x: 0})
IO.inspect x

In [ ]:
# Cromaの使用に関わらず、異なるStructどうしはマッチしない

%OtherStruct{x: x} = TestStruct.new!(%{x: 0})
# => MatchError

In [ ]:
# Structのフィールドをより詳細に定義する例

defmodule Food do
  defmodule Category do
    use Croma.SubtypeOfAtom,  values: [:meat, :vegitable, :fruit]
  end
  
  defmodule Name do
    use Croma.SubtypeOfString, pattern: ~r/\A.{1,50}\z/
  end
  
  use Croma.Struct, recursive_new?: true, fields: [
    category: Category,
    name:     Name
  ]
end


defmodule Eater do
  def eat(%{category: category, name: name}) do
    case category do
      :vegitable -> "I do not like #{name}, but I eat it..."
      _          -> "I love #{name}!"
    end
  end
end

In [ ]:
[
  %{category: :meat,      name: "pork"},
  %{category: :vegitable, name: "tomato"},
  %{category: :fruit,     name: "apple"}
]
|> Enum.map(&Food.new!/1)
|> Enum.map(&Eater.eat/1)

In [ ]:
# nameの文字数が50より大きい場合にはエラー
Food.new(%{category: :meat, name: "this meat is something having too long name and we cannot pronounce it"})

In [ ]:
# nameの文字数が0の場合にエラーになることを確かめよう

# categoryに未定義のatomが渡される場合もエラーになることを確かめよう


Cromaのマクロで関数の型スペックを簡潔に表したり、引数や返り値が期待した型であることのvalidationが可能。

- `def`に代わる`defun`
- `defp`に代わる`defunp`

In [ ]:
defmodule CromaTestModule do
  use Croma # Cromaが提供するマクロを利用するために必要
  
  @moduledoc """
  `defun`や`defunp`で引数に続けて`:: type`のように型を書く。
  `v[]`で型を囲むと、ランタイム時に引数の型が仕様にあっていることのvalidationが行われる。
  
  返り値も同様に表現する。
  
  `defun`や`defunp`で多重定義する際は、無名関数の多重定義のような書き方をする必要がある。
  """
  
  @spec add(integer, integer) :: integer
  def add(x, y) do
    x + y
  end
  
  defun add_integers(x :: v[integer], y :: v[integer]) :: v[integer] do
    x + y
  end
  
  defun add_integers_without_validation(x :: integer, y :: integer) :: integer do
    x + y
  end
  
  defun accept_hello_atom(value :: atom) :: Croma.Result.t(:hello, String.t) do
    (:hello)     -> {:ok, :hello}
    (other_atom) -> {:error, format_message(other_atom)}
  end
  
  defunp format_message(value :: v[atom]) :: v[String.t] do
    "#{value} is not :hello atom"
  end
end

In [ ]:
CromaTestModule.add_integers(0, 1)

In [ ]:
CromaTestModule.add_integers(0.0, 1)
# => %RuntimeError{message: "validation error: x is not a valid integer"}

In [ ]:
CromaTestModule.add_integers_without_validation(0.0, 1)

In [ ]:
IO.inspect CromaTestModule.accept_hello_atom(:hello)
IO.inspect CromaTestModule.accept_hello_atom(:world)

### meck

Erlang向けのモッキングライブラリ。

モジュールの関数の振る舞いを動的に自由に変えたり、呼び出し回数を計測したりできる。

In [ ]:
defmodule Hoge do
  def add(x, y) do
    x + y
  end
end

Hoge.add(1, 2)

In [ ]:
# Hogeモジュールをモック
# Erlangのライブラリなので、atomでmeckモジュールが表されている
:meck.new(Hoge, [:passthrough])

In [ ]:
# add関数の振る舞いを変える
# atomで関数名を指定することに注意
:meck.expect(Hoge, :add, fn(x, y) -> x + y + 2 end)

In [ ]:
Hoge.add(1, 2)

In [ ]:
:meck.num_calls(Hoge, :add, 2)

`:meck.expect`はテストの際によく使う。

- テスト中の現在日時を特定の値に固定する
- 外部との通信を行う関数で、通信が成功した場合と失敗した場合をそれぞれ想定

In [ ]:
# モックしたモジュールを削除
:meck.unload(Hoge)